In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
import os
import pandas as pd 
from ipyleaflet import GeoJSON, Marker, MarkerCluster

In [3]:
Map=geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

# 导入研究区和chla的采样点

In [ ]:
# 研究区
caohu = ee.FeatureCollection('users/311605001111/region/caohu_region')
Map.addLayer(caohu, {}, "caohu_vector")
Map.centerObject(caohu,10)   # 设置以研究区居中

In [ ]:
## 导入chla采样点的位置
# 导入点的shp文件
monitor_point = "D:/NDVI_exercise/caohu/monitor_point.shp"
# shp 转 ee
monitor_point_ee = geemap.shp_to_ee(monitor_point)
# 添加到layers
Map.addLayer(monitor_point_ee, {}, "monitor_point")

In [ ]:
# # 加载landsat8_SR 数据,并用研究区进行筛选
# landsat8=ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")

# # 去云掩膜函数
# def maskL8sr(image):
#   # Bits 3 and 5 are cloud shadow and cloud, respectively.
#   cloudShadowBitMask = (1 << 3)
#   cloudsBitMask = (1 << 5)
#   # Get the pixel QA band.
#   qa = image.select('pixel_qa')
#   # Both flags should be set to zero, indicating clear conditions.
#   mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
#                  .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
#   return image.updateMask(mask)

# def b5_b4(image):
#     B5_B4 = image.select('B5').divide(image.select('B4')).rename('B5_B4')
#     return image.addBands(B5_B4).clip(caohu)

# 提取chla点，对应的像元值作为csv文件

In [ ]:
# images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
#     .filterBounds(caohu) \
#     .filterDate('2016-01-01', '2016-01-31') \
#     .sort('CLOUD_COVER')

# best_image = images.first()

# print(best_image.get('system:id').getInfo())
# print(best_image.get('CLOUD_COVER').getInfo())

In [ ]:
caohu = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_121038_20160115').clip(caohu)
vis_params = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 6000,
  'gamma': 1.4,
}

caohu_image = caohu.addBands(caohu.select('B5').divide(caohu.select('B4')).rename('B5_B4'))

# Map.addLayer(caohu_image, vis_params, 'caohu_image')

In [ ]:
# # 添加数据文件所在目录的路径，idex_col指索引列
# df = pd.read_csv('D:/NDVI_exercise/caohu_chla.csv',index_col='id')

# maker_cluster = MarkerCluster(
#     markers=[Marker(location=[df.iloc[i,3],df.iloc[i,2]]) for i in range(0,19)],
#     name = 'Markers')
# Map.add_layer(maker_cluster)

In [ ]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
csv = os.path.join(out_dir, 'monitor_point.csv')

geemap.extract_values_to_points(monitor_point_ee,caohu_image,csv)

# 构建一元线性回归方程，y = wx+b。求系数w、b的值

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

In [4]:
# 读取上一步提取的csv文件
one_regression = pd.read_csv('C:/Users/DELL/Downloads/monitor_point.csv',index_col='id')

# 提取表格中相应列的值
chla = one_regression.loc[:,'chla'].values #叶绿素浓度
up_point = one_regression.loc[:,'B5_B4'].values #像元值

# 定义绘图的样式
def runplt():
    plt.figure()
    plt.title("Chla concentration inversion")
    plt.xlabel('up_point')
    plt.ylabel('chla')
    plt.grid(True)
    return plt

model = LinearRegression()  # 创建模型

X = up_point.reshape((-1,1))
y = chla

model.fit(X, y)  # 拟合

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/DELL/Downloads/monitor_point.csv'

## 绘图展示，数值的关系

In [ ]:
# 输出提取的数值
print(chla)
print(up_point)

# 根据提取的数字，绘图。x轴为up_point,y轴为chla
plt = runplt()
plt.plot(up_point, chla, 'k.')
plt.show()

## 拟合、预测

In [ ]:
# 进行预测
X2 = [[0.2], [0.4]] # 取两个预测值
y2 = model.predict(X2)  # 进行预测
print(y2)  # 查看预测值

plt = runplt()
plt.plot(up_point, chla, 'k.')
plt.plot(X2, y2, 'g-')  # 画出拟合曲线
plt.show()

## 求系数w、b的值，均方误差的值

In [ ]:
# 计算一元线性方程的系数，y = wx + b
variance = np.var(up_point, ddof=1)  # 计算方差，doff为贝塞尔（无偏估计）校正系数
covariance = np.cov(up_point, chla)[0][1]  # 计算协方差
w = covariance / variance
b = np.mean(chla) - w * np.mean(up_point)
print("w = %f\nb = %f" % (w, b))

chla_pred = w * up_point + b
plt = runplt()
plt.plot(up_point, chla, 'k.')  # 样本点
plt.plot(up_point, chla_pred, 'b-')  # 手动求出的线性回归模型
plt.show()

In [ ]:
# 计算均方误差
print("均方误差为: %f" % np.mean((model.predict(X)-y) ** 2))

#  根据系数w、b，建立模型，反演水质参数

In [ ]:
filename = os.path.join(out_dir, 'caohu.tif')

In [ ]:
# 将w,b带入一元线性方程：y = wx+b
caohu_chla = caohu_image.expression('w*(B5_B4)+b',{"w":w,'b':b,'B5_B4': caohu_image.select('B5_B4')}).rename('chla-concentration')
Map.addLayer(caohu_chla,{'palette': ['FFFFFF','00a000' ,'008000'], 'min': 0, 'max': 1.5}, 'caohu_chla')
Map

# 2016年月尺度产品的制作

In [ ]:
# 定义一些变量
days_31 = [1,3,5,7,8,10,12]
days_30 = [4,6,9,11]
days_28 = [2]

for i in range(2,3):
    if i in days_31:
        caohu_landsat=landsat8.filterDate('2016-{}-01'.format(i), '2016-{}-31'.format(i)).filterBounds(caohu).map(maskL8sr).map(b5_b4)
    elif i in days_30:
        caohu_landsat=landsat8.filterDate('2016-{}-01'.format(i), '2016-{}-30'.format(i)).filterBounds(caohu).map(maskL8sr).map(b5_b4)
    else:
        caohu_landsat=landsat8.filterDate('2016-{}-01'.format(i), '2016-{}-29'.format(i)).filterBounds(caohu).map(maskL8sr).map(b5_b4)
    caohu_monthmean = caohu_landsat.mean()
    caohu_chla = caohu_monthmean.expression('w*(B5_B4)+b',{"w":0.222075,'b':0.005561,'B5_B4': caohu_monthmean.select('B5_B4')}).rename('chla-concentration')
    Map.addLayer(caohu_chla,{'palette': ['FFFFFF','00a000' ,'008000'], 'min': 0, 'max': 1.5},'caohu_chla_{}月份'.format(i))
    geemap.ee_export_image_to_drive(caohu_chla, description='caohu_chla', folder='caohu_chla',scale=30)